In [253]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('high_freq.db')
df = pd.read_sql("select z.*, s.Security_name from zhangtinggu z join Security_name s on z.thscode = s.thscode where z.thscode like '0%'",conn)
print(df)
conn.close()


                     time    thscode   open   high    low  close   avgPrice  \
0        2023-11-01 09:30  002045.SZ  15.74  15.74  15.74  15.74  15.740000   
1        2023-11-01 09:31  002045.SZ  15.69  15.75  15.50  15.52  15.654700   
2        2023-11-01 09:32  002045.SZ  15.52  15.60  15.52  15.60  15.554100   
3        2023-11-01 09:33  002045.SZ  15.61  15.68  15.54  15.67  15.590900   
4        2023-11-01 09:34  002045.SZ  15.67  15.67  15.59  15.64  15.636500   
...                   ...        ...    ...    ...    ...    ...        ...   
3325072  2023-09-19 14:56  002173.SZ   8.09   8.10   8.09   8.09   8.096483   
3325073  2023-09-19 14:57  002173.SZ   8.10   8.10   8.09   8.09   8.098415   
3325074  2023-09-19 14:58  002173.SZ   8.10   8.10   8.10   8.10   8.100000   
3325075  2023-09-19 14:59  002173.SZ   8.10   8.10   8.10   8.10        NaN   
3325076  2023-09-19 15:00  002173.SZ   8.09   8.09   8.09   8.09   8.090000   

           volume      amount  change  changeRatio 

In [254]:
tmp=df.sort_values(['time','thscode'],ascending=True)
print(tmp)

                     time    thscode   open   high    low  close  avgPrice  \
1185479  2023-08-21 09:30  000004.SZ  16.64  16.64  16.64  16.64   16.6400   
1185720  2023-08-21 09:30  000004.SZ  16.64  16.64  16.64  16.64   16.6400   
1411537  2023-08-21 09:30  000010.SZ   3.11   3.11   3.11   3.11    3.1100   
1411778  2023-08-21 09:30  000010.SZ   3.11   3.11   3.11   3.11    3.1100   
1614941  2023-08-21 09:30  000023.SZ   6.77   6.77   6.77   6.77    6.7700   
...                   ...        ...    ...    ...    ...    ...       ...   
1615663  2023-11-01 15:00  003020.SZ  30.93  30.93  30.93  30.93   30.9300   
1615904  2023-11-01 15:00  003020.SZ  30.93  30.93  30.93  30.93   30.9298   
2950803  2023-11-01 15:00  003032.SZ  13.45  13.45  13.45  13.45   13.4500   
2951044  2023-11-01 15:00  003032.SZ  13.45  13.45  13.45  13.45   13.4500   
623707   2023-11-01 15:00  003040.SZ  17.90  17.90  17.90  17.90   17.9000   

            volume      amount  change  changeRatio  turnoverRa

In [255]:
df = df.drop_duplicates(subset=['time', 'thscode'], keep='first')


df = df.drop_duplicates(subset=['time', 'thscode'], keep='first')
df['time'] = pd.to_datetime(df['time'])
# 然后从'time'列提取日期，并创建新的'date'列
df['date'] = df['time'].dt.date


print(df)

                       time    thscode   open   high    low  close   avgPrice  \
0       2023-11-01 09:30:00  002045.SZ  15.74  15.74  15.74  15.74  15.740000   
1       2023-11-01 09:31:00  002045.SZ  15.69  15.75  15.50  15.52  15.654700   
2       2023-11-01 09:32:00  002045.SZ  15.52  15.60  15.52  15.60  15.554100   
3       2023-11-01 09:33:00  002045.SZ  15.61  15.68  15.54  15.67  15.590900   
4       2023-11-01 09:34:00  002045.SZ  15.67  15.67  15.59  15.64  15.636500   
...                     ...        ...    ...    ...    ...    ...        ...   
3324831 2023-09-19 14:56:00  002173.SZ   8.09   8.10   8.09   8.09   8.096483   
3324832 2023-09-19 14:57:00  002173.SZ   8.10   8.10   8.09   8.09   8.098415   
3324833 2023-09-19 14:58:00  002173.SZ   8.10   8.10   8.10   8.10   8.100000   
3324834 2023-09-19 14:59:00  002173.SZ   8.10   8.10   8.10   8.10        NaN   
3324835 2023-09-19 15:00:00  002173.SZ   8.09   8.09   8.09   8.09   8.090000   

           volume      amou

In [259]:
#峰度和偏度#


# 确保DataFrame是按照thscode和date排序的
df = df.sort_values(by=['thscode', 'date'])


# 计算对数收益率，但首先排除每天的第一条记录
df['log_return'] = np.where(
    df['time'].dt.normalize() != df['time'].shift(1).dt.normalize(), 
    np.nan,  # 如果是新的一天，则不计算对数收益率，赋值为NaN
    np.log(df['close'] / df['close'].shift(1))  # 否则，正常计算对数收益率
)

# 通过groupby计算每日的偏度和峰度
daily_skewness = df.groupby(['thscode', 'date'])['log_return'].skew().rename('daily_skewness')
daily_kurtosis = df.groupby(['thscode', 'date'])['log_return'].apply(pd.Series.kurt).rename('daily_kurtosis')
daily_variance = df.groupby(['thscode', 'date'])['log_return'].var().rename('daily_variance')

daily_skewness = daily_skewness.reset_index()
daily_kurtosis = daily_kurtosis.reset_index()
daily_variance = daily_variance.reset_index()

# 将计算结果合并回原始DataFrame
df = df.merge(daily_skewness, on=['thscode', 'date'], how='left')
df = df.merge(daily_kurtosis, on=['thscode', 'date'], how='left')
df = df.merge(daily_variance, on=['thscode', 'date'], how='left')

print(df)

                       time    thscode   open   high    low  close   avgPrice  \
0       2023-08-21 09:30:00  000004.SZ  16.64  16.64  16.64  16.64  16.640000   
1       2023-08-21 09:31:00  000004.SZ  16.62  16.85  16.52  16.85  16.672619   
2       2023-08-21 09:32:00  000004.SZ  16.88  16.94  16.80  16.80  16.822798   
3       2023-08-21 09:33:00  000004.SZ  16.81  16.95  16.80  16.93  16.896727   
4       2023-08-21 09:34:00  000004.SZ  16.93  17.03  16.90  17.03  16.949838   
...                     ...        ...    ...    ...    ...    ...        ...   
1667233 2023-11-01 14:56:00  003040.SZ  17.86  17.90  17.86  17.90  17.876800   
1667234 2023-11-01 14:57:00  003040.SZ  17.90  17.91  17.88  17.90  17.893700   
1667235 2023-11-01 14:58:00  003040.SZ  17.89  17.89  17.89  17.89  17.890900   
1667236 2023-11-01 14:59:00  003040.SZ  17.89  17.89  17.89  17.89   0.000000   
1667237 2023-11-01 15:00:00  003040.SZ  17.90  17.90  17.90  17.90  17.900000   

           volume     amoun

In [261]:
# 定义计算Pearson相关系数的函数
def pearson_correlation(group):
    correlation = group['close'].corr(group['volume'])
    group['pearson_corr'] = correlation
    return group

# 按股票代码和日期分组并应用函数，不添加分组键到索引
df = df.groupby(['thscode', 'date']).apply(pearson_correlation)

# 重置索引，如果你有多级索引的话
df = df.reset_index(drop=True)

print(df)

                       time    thscode   open   high    low  close   avgPrice  \
0       2023-08-21 09:30:00  000004.SZ  16.64  16.64  16.64  16.64  16.640000   
1       2023-08-21 09:31:00  000004.SZ  16.62  16.85  16.52  16.85  16.672619   
2       2023-08-21 09:32:00  000004.SZ  16.88  16.94  16.80  16.80  16.822798   
3       2023-08-21 09:33:00  000004.SZ  16.81  16.95  16.80  16.93  16.896727   
4       2023-08-21 09:34:00  000004.SZ  16.93  17.03  16.90  17.03  16.949838   
...                     ...        ...    ...    ...    ...    ...        ...   
1667233 2023-11-01 14:56:00  003040.SZ  17.86  17.90  17.86  17.90  17.876800   
1667234 2023-11-01 14:57:00  003040.SZ  17.90  17.91  17.88  17.90  17.893700   
1667235 2023-11-01 14:58:00  003040.SZ  17.89  17.89  17.89  17.89  17.890900   
1667236 2023-11-01 14:59:00  003040.SZ  17.89  17.89  17.89  17.89   0.000000   
1667237 2023-11-01 15:00:00  003040.SZ  17.90  17.90  17.90  17.90  17.900000   

           volume     amoun

In [262]:
#成交量分布因子


import pandas as pd

# 确保 'time' 列是日期时间格式
df['time'] = pd.to_datetime(df['time'])

# 创建时间段边界
morning_times = pd.date_range(start='09:30:00', end='11:30:00', periods=5).time
afternoon_times = pd.date_range(start='13:00:00', end='15:00:00', periods=5).time

# 分配时间段
def assign_time_bin(row_time):
    for i in range(4):  # 注意这里的范围是4，因为我们有5个时间点，形成4个时间段
        if morning_times[i] <= row_time <= morning_times[i+1]:
            return 'M' + str(i+1)
        elif afternoon_times[i] <= row_time <= afternoon_times[i+1]:
            return 'A' + str(i+1)
    return 'Out_of_session'

# 应用时间段分配
df['time_bin'] = df['time'].dt.time.apply(assign_time_bin)

# 定义函数来计算每只股票每天的成交量分布因子
def calculate_volume_distribution_factor(group):
    volume_by_bin = group.groupby('time_bin')['volume'].sum()
    total_volume = group['volume'].sum()
    volume_distribution_factor = volume_by_bin / total_volume
    # 映射分布因子回group
    group['volume_distribution_factor'] = group['time_bin'].map(volume_distribution_factor)
    return group

# 对每只股票每天的数据应用函数并合并结果
df = df.groupby(['thscode', 'date']).apply(calculate_volume_distribution_factor).reset_index(drop=True)


print(df)


                       time    thscode   open   high    low  close   avgPrice  \
0       2023-08-21 09:30:00  000004.SZ  16.64  16.64  16.64  16.64  16.640000   
1       2023-08-21 09:31:00  000004.SZ  16.62  16.85  16.52  16.85  16.672619   
2       2023-08-21 09:32:00  000004.SZ  16.88  16.94  16.80  16.80  16.822798   
3       2023-08-21 09:33:00  000004.SZ  16.81  16.95  16.80  16.93  16.896727   
4       2023-08-21 09:34:00  000004.SZ  16.93  17.03  16.90  17.03  16.949838   
...                     ...        ...    ...    ...    ...    ...        ...   
1667233 2023-11-01 14:56:00  003040.SZ  17.86  17.90  17.86  17.90  17.876800   
1667234 2023-11-01 14:57:00  003040.SZ  17.90  17.91  17.88  17.90  17.893700   
1667235 2023-11-01 14:58:00  003040.SZ  17.89  17.89  17.89  17.89  17.890900   
1667236 2023-11-01 14:59:00  003040.SZ  17.89  17.89  17.89  17.89   0.000000   
1667237 2023-11-01 15:00:00  003040.SZ  17.90  17.90  17.90  17.90  17.900000   

           volume     amoun

In [263]:
import pandas as pd
import numpy as np


# 计算每只股票每天的成交量变化量
df['volume_change'] = df.groupby(['thscode', 'date'])['volume'].diff().fillna(0)

# 定义指示函数
def indicator(x, direction):
    if direction == 'up':
        return x if x > 0 else 0
    elif direction == 'down':
        return x if x < 0 else 0
    else:
        return 0

# 应用指示函数并计算平方
df['vol_up_squared'] = df['volume_change'].apply(lambda x: indicator(x, 'up')) ** 2
df['vol_down_squared'] = df['volume_change'].apply(lambda x: indicator(x, 'down')) ** 2

# 计算每只股票每天的指标
def calculate_daily_metrics(group):
    volume_upward_volatility = np.sqrt(group['vol_up_squared'].sum())
    volume_downward_volatility = np.sqrt(group['vol_down_squared'].sum())
    total_vol_squared = group['volume_change'].pow(2).sum()
    volume_upward_volatility_ratio = volume_upward_volatility / np.sqrt(total_vol_squared)
    volume_downward_volatility_ratio = volume_downward_volatility / np.sqrt(total_vol_squared)
    
    # 返回一个包含所有计算指标的Series
    return pd.Series({
        'volume_upward_volatility': volume_upward_volatility,
        'volume_downward_volatility': volume_downward_volatility,
        'volume_upward_volatility_ratio': volume_upward_volatility_ratio,
        'volume_downward_volatility_ratio': volume_downward_volatility_ratio
    })

# 应用计算函数并创建一个新的DataFrame
metrics_df = df.groupby(['thscode', 'date']).apply(calculate_daily_metrics).reset_index()

# 将计算结果合并回原始DataFrame
df = df.merge(metrics_df, on=['thscode', 'date'], how='left')

print(df.head())


                 time    thscode   open   high    low  close   avgPrice  \
0 2023-08-21 09:30:00  000004.SZ  16.64  16.64  16.64  16.64  16.640000   
1 2023-08-21 09:31:00  000004.SZ  16.62  16.85  16.52  16.85  16.672619   
2 2023-08-21 09:32:00  000004.SZ  16.88  16.94  16.80  16.80  16.822798   
3 2023-08-21 09:33:00  000004.SZ  16.81  16.95  16.80  16.93  16.896727   
4 2023-08-21 09:34:00  000004.SZ  16.93  17.03  16.90  17.03  16.949838   

     volume     amount  change  changeRatio  turnoverRatio  sellVolume  \
0  164200.0  2732288.0   -0.01    -0.060060       0.130020         0.0   
1  291700.0  4863403.0    0.21     1.262019       0.230980         0.0   
2  339500.0  5711340.0   -0.05    -0.296736       0.268830         0.0   
3  299400.0  5058880.0    0.13     0.773810       0.237077         0.0   
4  278400.0  4718835.0    0.10     0.590667       0.220448         0.0   

   buyVolume        BBI      MA      MACD      TRIX        RSI        ARBR  \
0        0.0  16.694063  1

In [264]:
print(len(df['volume_distribution_factor'].unique()))

55316


In [265]:
df

,time,thscode,open,high,low,close,avgPrice,volume,amount,change,changeRatio,turnoverRatio,sellVolume,buyVolume,BBI,MA,MACD,TRIX,RSI,ARBR,BOLL,VOSC,ZDZB,SECURITY_NAME,date,log_return,daily_skewness,daily_kurtosis,daily_variance,pearson_corr,time_bin,volume_distribution_factor,volume_change,vol_up_squared,vol_down_squared,volume_upward_volatility,volume_downward_volatility,volume_upward_volatility_ratio,volume_downward_volatility_ratio
0,2023-08-21 09:30:00,000004.SZ,16.64,16.64,16.64,16.64,16.640000,164200.0,2732288.0,-0.01,-0.060060,0.130020,0.0,0.0,16.694063,16.650,-0.038941,-0.024234,14.903698,115.000000,16.624490,19.866491,0.876979,国华网安,2023-08-21,NaN,1.632163,8.968361,0.000005,0.294682,M1,0.49380,0.0,0.000000e+00,0.000000e+00,4.610838e+05,4.027191e+05,0.753167,0.65783
1,2023-08-21 09:31:00,000004.SZ,16.62,16.85,16.52,16.85,16.672619,291700.0,4863403.0,0.21,1.262019,0.230980,0.0,0.0,16.718854,16.688,-0.024281,-0.023917,75.995118,146.808511,16.622663,22.313232,0.888460,国华网安,2023-08-21,0.012541,1.632163,8.968361,0.000005,0.294682,M1,0.49380,127500.0,1.625625e+10,0.000000e+00,4.610838e+05,4.027191e+05,0.753167,0.65783
2,2023-08-21 09:32:00,000004.SZ,16.88,16.94,16.80,16.80,16.822798,339500.0,5711340.0,-0.05,-0.296736,0.268830,0.0,0.0,16.737813,16.718,-0.016508,-0.021499,63.060369,132.727273,16.624381,27.604343,0.899941,国华网安,2023-08-21,-0.002972,1.632163,8.968361,0.000005,0.294682,M1,0.49380,47800.0,2.284840e+09,0.000000e+00,4.610838e+05,4.027191e+05,0.753167,0.65783
3,2023-08-21 09:33:00,000004.SZ,16.81,16.95,16.80,16.93,16.896727,299400.0,5058880.0,0.13,0.773810,0.237077,0.0,0.0,16.778646,16.774,0.000141,-0.015058,75.872844,156.363636,16.614916,29.159359,0.917432,国华网安,2023-08-21,0.007708,1.632163,8.968361,0.000005,0.294682,M1,0.49380,-40100.0,0.000000e+00,1.608010e+09,4.610838e+05,4.027191e+05,0.753167,0.65783
4,2023-08-21 09:34:00,000004.SZ,16.93,17.03,16.90,17.03,16.949838,278400.0,4718835.0,0.10,0.590667,0.220448,0.0,0.0,16.817917,16.850,0.021160,-0.004362,81.724182,162.068966,16.593759,31.248467,0.929269,国华网安,2023-08-21,0.005889,1.632163,8.968361,0.000005,0.294682,M1,0.49380,-21000.0,0.000000e+00,4.410000e+08,4.610838e+05,4.027191e+05,0.753167,0.65783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667233,2023-11-01 14:56:00,003040.SZ,17.86,17.90,17.86,17.90,17.876800,231600.0,4140270.0,0.03,0.167900,0.106900,85400.0,145700.0,17.878229,17.864,-0.026708,-0.026631,65.396494,104.000000,17.826538,30.948152,1.049180,楚天龙,2023-11-01,0.001677,-0.121127,2.762032,0.000003,0.065137,A4,0.18849,50500.0,2.550250e+09,0.000000e+00,1.099452e+06,1.080492e+06,0.713230,0.70093
1667234,2023-11-01 14:57:00,003040.SZ,17.90,17.91,17.88,17.90,17.893700,224500.0,4017130.0,0.00,0.000000,0.103700,97200.0,127000.0,17.883125,17.874,-0.023086,-0.024674,65.396494,96.296296,17.827250,30.850904,1.056011,楚天龙,2023-11-01,0.000000,-0.121127,2.762032,0.000003,0.065137,A4,0.18849,-7100.0,0.000000e+00,5.041000e+07,1.099452e+06,1.080492e+06,0.713230,0.70093
1667235,2023-11-01 14:58:00,003040.SZ,17.89,17.89,17.89,17.89,17.890900,3300.0,59040.0,-0.01,-0.055900,0.001500,1100.0,2100.0,17.886354,17.882,-0.020783,-0.022538,56.099031,92.592593,17.827908,22.442506,1.062842,楚天龙,2023-11-01,-0.000559,-0.121127,2.762032,0.000003,0.065137,A4,0.18849,-221200.0,0.000000e+00,4.892944e+10,1.099452e+06,1.080492e+06,0.713230,0.70093
1667236,2023-11-01 14:59:00,003040.SZ,17.89,17.89,17.89,17.89,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,17.886875,17.890,-0.018742,-0.020348,56.099031,88.888889,17.829280,15.550470,1.076734,楚天龙,2023-11-01,0.000000,-0.121127,2.762032,0.000003,0.065137,A4,0.18849,-3300.0,0.000000e+00,1.089000e+07,1.099452e+06,1.080492e+06,0.713230,0.70093


In [266]:

# 首先，计算每只股票每天的成交量变化量
df['volume_change'] = df.groupby(['thscode', 'date'])['volume'].diff().fillna(0)

# 接下来，计算每只股票每天的成交量上行波动性和下行波动性
df['vol_up_squared'] = np.where(df['volume_change'] > 0, df['volume_change']**2, 0)
df['vol_down_squared'] = np.where(df['volume_change'] < 0, df['volume_change']**2, 0)

# 每只股票每天的波动性指标
volatility_metrics = df.groupby(['thscode', 'date']).agg(
    volume_upward_volatility=pd.NamedAgg(column='vol_up_squared', aggfunc=lambda x: np.sqrt(x.sum())),
    volume_downward_volatility=pd.NamedAgg(column='vol_down_squared', aggfunc=lambda x: np.sqrt(x.sum()))
)

# 计算成交量上行和下行波动性比
total_vol_squared = df.groupby(['thscode', 'date'])['volume_change'].apply(lambda x: (x**2).sum())
volatility_metrics['volume_upward_volatility_ratio'] = volatility_metrics['volume_upward_volatility'] / np.sqrt(total_vol_squared)
volatility_metrics['volume_downward_volatility_ratio'] = volatility_metrics['volume_downward_volatility'] / np.sqrt(total_vol_squared)

# 重新创建result DataFrame
result = df.groupby(['thscode', 'date']).agg(
    daily_skewness=('daily_skewness', 'mean'),
    daily_kurtosis=('daily_kurtosis', 'mean'),
    daily_variance=('daily_variance', 'mean'),
    pearson_corr=('pearson_corr', 'mean')
).reset_index()

# 合并波动性指标
result = result.merge(volatility_metrics, on=['thscode', 'date'], how='left')


print(result)


        thscode        date  daily_skewness  daily_kurtosis  daily_variance  \
0     000004.SZ  2023-08-21        1.632163        8.968361        0.000005   
1     000004.SZ  2023-08-22        1.547498        8.702269        0.000004   
2     000004.SZ  2023-08-23        4.197470       45.139436        0.000015   
3     000004.SZ  2023-08-24        1.445023        7.397598        0.000010   
4     000004.SZ  2023-08-25       -1.297136       23.930925        0.000012   
...         ...         ...             ...             ...             ...   
6913  003040.SZ  2023-10-26        2.311745       17.684372        0.000011   
6914  003040.SZ  2023-10-27        1.649165        8.203461        0.000007   
6915  003040.SZ  2023-10-30        0.793026        3.287886        0.000004   
6916  003040.SZ  2023-10-31        3.730938       25.368349        0.000006   
6917  003040.SZ  2023-11-01       -0.121127        2.762032        0.000003   

      pearson_corr  volume_upward_volatility  volum

In [267]:

# 使用 pivot_table 将 'volume_distribution_factor' 分布按 'time_bin' 重塑成多列
time_bin_vdf = df.pivot_table(index=['thscode', 'date'], 
                              columns='time_bin', 
                              values='volume_distribution_factor', 
                              aggfunc='mean').reset_index()

# 将列索引简化，用 'time_bin' 的值作为新列名的前缀
time_bin_vdf.columns = [str(col) if col == 'thscode' or col == 'date' else f'time_bin_{col}' for col in time_bin_vdf.columns]

# 将这些 'time_bin' 列合并到 result DataFrame 中
result = pd.merge(result, time_bin_vdf, on=['thscode', 'date'], how='left')


result

,thscode,date,daily_skewness,daily_kurtosis,daily_variance,pearson_corr,volume_upward_volatility,volume_downward_volatility,volume_upward_volatility_ratio,volume_downward_volatility_ratio,time_bin_A1,time_bin_A2,time_bin_A3,time_bin_A4,time_bin_M1,time_bin_M2,time_bin_M3,time_bin_M4
0,000004.SZ,2023-08-21,1.632163,8.968361,0.000005,0.294682,4.610838e+05,4.027191e+05,0.753167,0.657830,0.054933,0.059586,0.080318,0.125032,0.493800,0.079928,0.068241,0.038163
1,000004.SZ,2023-08-22,1.547498,8.702269,0.000004,0.491875,5.907764e+05,4.430421e+05,0.800026,0.599965,0.057941,0.055948,0.237730,0.221722,0.186214,0.131757,0.065218,0.043470
2,000004.SZ,2023-08-23,4.197470,45.139436,0.000015,0.171985,1.180099e+06,8.753989e+05,0.803150,0.595777,0.048471,0.038868,0.068469,0.152740,0.154283,0.389039,0.102658,0.045471
3,000004.SZ,2023-08-24,1.445023,7.397598,0.000010,0.447958,6.936484e+05,6.053671e+05,0.753424,0.657535,0.072351,0.033978,0.054979,0.159800,0.425873,0.085955,0.125682,0.041382
4,000004.SZ,2023-08-25,-1.297136,23.930925,0.000012,0.503549,7.222070e+05,6.977789e+05,0.719165,0.694840,0.050146,0.066213,0.042563,0.090724,0.355180,0.192235,0.144538,0.058401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6913,003040.SZ,2023-10-26,2.311745,17.684372,0.000011,0.087345,1.613498e+06,1.251740e+06,0.790112,0.612963,0.055921,0.063637,0.129244,0.140548,0.316027,0.071330,0.153443,0.069850
6914,003040.SZ,2023-10-27,1.649165,8.203461,0.000007,0.049403,1.185058e+06,8.054668e+05,0.827047,0.562132,0.045896,0.195749,0.073668,0.159046,0.287053,0.125925,0.061113,0.051550
6915,003040.SZ,2023-10-30,0.793026,3.287886,0.000004,-0.090769,8.612434e+05,7.102241e+05,0.771506,0.636222,0.062008,0.114362,0.066276,0.149884,0.333510,0.154307,0.061375,0.058279
6916,003040.SZ,2023-10-31,3.730938,25.368349,0.000006,0.111290,1.580645e+06,1.177276e+06,0.801995,0.597331,0.076801,0.063177,0.059763,0.120456,0.167151,0.049531,0.351380,0.111741
